In [4]:
import streamlit as st
from streamlit_option_menu import option_menu
import easyocr
from PIL import Image
import pandas as pd
import numpy as np
import psycopg2 as sql
import re
import io

In [5]:
def img_pro(img):
    img = Image.open(img)
    img_arr = np.array(img)
    reader = easyocr.Reader(['en'])
    txt = reader.readtext(img_arr, detail=0)
    data=txt
    extracted={'Name':[],'Designation':[],'Ph_No':[],'website':[],'Address':[],'pincode':[],'company_name':[],'mail':[]}
    extracted['Name'].append(data[0])
    extracted['Designation'].append(data[1])
    for i in range (len(data)):
        if data[i].startswith("+") or (data[i].replace('-','').isdigit() and '-' in data[i]):
            extracted['Ph_No'].append(data[i])
        
        elif 'TamilNadu' in data[i] or 'Tamil Nadu' in data[i] or data[i].isdigit():
            extracted['pincode'].append(data[i])
            
        elif data[i].startswith('www') or data[i].startswith('WWW') or data[i].startswith('wwW'):
            extracted['website'].append(data[i])
            
        elif '@'in data[i] and '.com' in data[i]:
            extracted['mail'].append(data[i])
            
        elif re.match(r'[A-Za-z]',data[i]):
            extracted['company_name'].append(data[i])
        
        
        elif 'st' in data[i] or 'ST' in data[i] or 'St' in data[i] or 'sT' in data[i]:
            extracted['Address'].append(data[i])
    
    for key,value in extracted.items():
        if len(value)>0:
            concad=" ".join(value)
            extracted[key]=[concad]
        else:
            value="NA"
            extracted[key]=[value]
    img_byt = io.BytesIO()
    img.save(img_byt, format="PNG")
    img_data = img_byt.getvalue()
    extracted['IMAGE']=(img_data)
       
    return extracted

In [6]:
img = r"C:\Users\krish\OneDrive\Desktop\project\ocr\5.png"
dta=img_pro(img)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [7]:
dta

{'Name': ['SANTHOSH'],
 'Designation': ['Technical Manager'],
 'Ph_No': ['+123-456-1234'],
 'website': ['www.Suncom'],
 'Address': ['NA'],
 'pincode': ['123 ABC St , Tirupur, TamilNadu; 641603'],
 'company_name': ['SANTHOSH Technical Manager Sun Electricals'],
 'mail': ['hello@Sun.com'],
 'IMAGE': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\xec\x00\x00\x02N\x08\x06\x00\x00\x00-z\x8d\xd9\x00\x01\x00\x00IDATx\x9c\xec\x9du\x9c\x1c\xc7\x99\xbf\x9f\xea\xee\xc1\xe5]1Z\xb2$\xcb\xb6\xcc\xcc\x1cS\xc0\x8ec\'1\xe5\x82\x17\xbc\\\xf8\xee\x17\xbc\xbb0\\rafp\xc01\xc7133K\xb2\x98\xa5e\x1aj\xa8\xdf\x1f\xd5\xdd3\xb3 \xedJ+i%\xbd\xcf\'\x13yg\xba\xab\xab\xab\xabg\xfa[/)\xad\xb5F\x10\x04A\x10\x04A\x10\x04t\x10\xa0,\x0b\xaf}-]?\xff \xf9Go\x04\x0b\x94\xe3\x98\xcf\xb5F)\xb5\x9b{)\x08\xc2\xdeJ$\xcf\x9d\x96\x19L\xf9\xe1J\xac\xdd\xdc\x1fA\x10\x04A\x10\x04A\x187(\xcbBk\x8d\xdd<\x83\t\x9f\xba\x81\xba\xd7\xff;\xcaq\xd0\xae\x17\x8bu\xb1w\t\x82\xb0\xab\x10\xc1.\x08\x82 \x08\x82 \x08!\x95\x16\xf4\xa0\xd0O\xe3\xd

In [8]:
df=pd.DataFrame(dta)
df

,Name,Designation,Ph_No,website,Address,pincode,company_name,mail,IMAGE
0,SANTHOSH,Technical Manager,+123-456-1234,www.Suncom,NA,"123 ABC St , Tirupur, TamilNadu; 641603",SANTHOSH Technical Manager Sun Electricals,hello@Sun.com,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...


In [15]:
connection=sql.connect(host='localhost',password='vignesh',dbname='img_to_txt',user='postgres',port=5432)
my_corsor=connection.cursor()

In [47]:
def in_sql(df):
    connection=sql.connect(host='localhost',password='vignesh',dbname='img_to_txt',user='postgres',port=5432)
    my_corsor=connection.cursor()
    cr_query='''create table if not exists img (Name varchar(50), Designation varchar(50),Ph_No varchar(50),website varchar(100),Address varchar(150),pincode varchar,company_name varchar,mail varchar,image_byte varchar)'''
    my_corsor.execute(cr_query)
    for i,j in df.iterrows():
        in_query='''insert into img (Name,Designation,Ph_No,website,Address,pincode,company_name,mail,image_byte) values (%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
        values=(j['Name'],j['Designation'],j['Ph_No'][:25],j['website'],j['Address'],j['pincode'],j['company_name'],j['mail'],j['IMAGE'])
        my_corsor.execute(in_query,values)
    connection.commit()

In [17]:
def img_pro(img):
    img = Image.open(img)
    img_arr = np.array(img)
    reader = easyocr.Reader(['en'])
    txt = reader.readtext(img_arr, detail=0)
    data=txt
    extracted={'Name':[],'Designation':[],'Ph_No':[],'website':[],'Address':[],'pincode':[],'company_name':[],'mail':[]}
    extracted['Name'].append(data[0])
    extracted['Designation'].append(data[1])
    for i in range (len(data)):
        if data[i].startswith("+") or (data[i].replace('-','').isdigit() and '-' in data[i]):
            extracted['Ph_No'].append(data[i])
        
        elif 'TamilNadu' in data[i] or 'Tamil Nadu' in data[i] or data[i].isdigit():
            extracted['pincode'].append(data[i])
            
        elif data[i].startswith('www') or data[i].startswith('WWW') or data[i].startswith('wwW'):
            extracted['website'].append(data[i])
            
        elif '@'in data[i] and '.com' in data[i]:
            extracted['mail'].append(data[i])
            
        elif re.match(r'[A-Za-z]',data[i]):
            extracted['company_name'].append(data[i])
        
        
        elif 'st' in data[i] or 'ST' in data[i] or 'St' in data[i] or 'sT' in data[i]:
            extracted['Address'].append(data[i])
    
    for key,value in extracted.items():
        if len(value)>0:
            concad=" ".join(value)
            extracted[key]=[concad]
        else:
            value="NA"
            extracted[key]=[value]
    img_byt = io.BytesIO()
    img.save(img_byt, format="PNG")
    img_data = img_byt.getvalue()
    extracted['IMAGE']=(img_data)
    df=pd.DataFrame(extracted)
    df.index=(range(1,len(df)+1))
    st.write(df)
    return df

In [35]:
img = r"C:\Users\krish\OneDrive\Desktop\project\ocr\1.png"
img_pro(img)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


,Name,Designation,Ph_No,website,Address,pincode,company_name,mail,IMAGE
1,Selva,DATA MANAGER,+123-456-7890 +123-456-7891,WWW XYZI.com,"123 ABC St , Chennai;",TamilNadu 600113,Selva DATA MANAGER selva digitals,hello@XYZ1.com,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...


In [39]:
import psycopg2 as sql  # Import psycopg2 for PostgreSQL

def in_sql(df):
    # Establish a connection to the database
    connection = sql.connect(host='localhost', password='vignesh', dbname='img_to_txt', user='postgres', port=5432)
    
    # Create a cursor object
    my_cursor = connection.cursor()

    # Create the table if it doesn't exist
    cr_query = '''CREATE TABLE IF NOT EXISTS img (
                   Name VARCHAR(255), Designation VARCHAR(255), Ph_No VARCHAR(50), 
                   website text, Address text, pincode VARCHAR(50), 
                   company_name VARCHAR(100), mail text, image_byte VARCHAR(500))'''
    my_cursor.execute(cr_query)

    # Insert data from the DataFrame into the table
    for i, j in df.iterrows():
        try:
            in_query = '''INSERT INTO img 
                          (Name, Designation, Ph_No, website, Address, pincode, company_name, mail, image_byte) 
                          VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'''
            values = (j['Name'], j['Designation'], j['Ph_No'], j['website'], j['Address'], j['pincode'],
                      j['company_name'], j['mail'], j['IMAGE'])
            my_cursor.execute(in_query, values)
        except sql.Error as e:
            print('Error:', e)
            connection.rollback()  # Rollback the transaction in case of an error

    # Commit the transaction
    connection.commit()

    # Close the cursor and connection
    my_cursor.close()
    connection.close()


In [48]:
img = r"C:\Users\krish\OneDrive\Desktop\project\ocr\1.png"
df=img_pro(img)
in_sql(df)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


C:\Users\krish\AppData\Local\Temp\ipykernel_34132\2819979457.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql_query("select * from img",connection)


3